In [1]:
import pymysql.cursors

In [4]:
# DB 연결
connection = pymysql.connect(host='localhost', # IP
                             user='root', # 계정
                             password='songdy0123', # 패스워드
                             db='crawling', # 데이터베이스 스키마 이름
                             charset='utf8', # default-characgerset
                             cursorclass=pymysql.cursors.DictCursor)

In [15]:
# DB 데이터 삽입: sql문으로 진행
with connection.cursor() as cursor:
    sql = "INSERT INTO `source` (`id`, `name`, `url`) VALUES (%s, %s, %s)"
    cursor.execute(sql, (10, '트위터', 'http://twitter.com'))
connection.commit()

**실행 결과**

![db-twitter](./images/db_1.png)

In [16]:
source_id = cursor.lastrowid
source_id

10

In [17]:
type(source_id)

int

< 연습 >

 crawl_data 테이블에 임의의 데이터를 추가하시오. 단, source_id는 방금 추가한 source 데이터로 한다.

In [23]:
with connection.cursor() as cursor:
    sql = "INSERT INTO `data` (`data_id`, `contents`, `src_id`, `url`)"
    sql += " VALUES (%s, %s, %s, %s)"
    cursor.execute(sql, (1, '임시 데이터', source_id,
                        'https://search.naver.com/search.naver?query=무역전쟁'))
connection.commit()

In [24]:
# 데이터 추가 확인
with connection.cursor() as cursor:
    sql = "SELECT * FROM `data`"
    cursor.execute(sql)
    data = cursor.fetchall()
    print(data)

[{'data_id': 1, 'contents': '임시 데이터', 'src_id': 10, 'url': 'https://search.naver.com/search.naver?query=무역전쟁'}]


**실행 결과**
![db-temp](./images/db_2.png)

```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-12-6733754fae30> in <module>
      1 with connection.cursor() as cursor:
      2     sql = "INSERT INTO `DATA` (`DATA_ID`, `CONTENTS`, `SRC_ID`, `URL`) VALUES (%d, %s, %d, %s)"
----> 3     cursor.execute(sql, (source_id, 'Hello, Elon Musk @ElonMusk', source_id, 'http://twitter.com'))
      4 connection.commit()

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\cursors.py in execute(self, query, args)
    166             pass
    167 
--> 168         query = self.mogrify(query, args)
    169 
    170         result = self._query(query)

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\cursors.py in mogrify(self, query, args)
    145 
    146         if args is not None:
--> 147             query = query % self._escape_args(args, conn)
    148 
    149         return query

TypeError: %d format: a number is required, not str
```

In [11]:
source_id, type(source_id)

(1, int)

```
---------------------------------------------------------------------------
InternalError                             Traceback (most recent call last)
<ipython-input-22-ac482f564185> in <module>
      3     sql += " VALUES (%s, %s, %s)"
      4     cursor.execute(sql, ('임시 데이터', source_id,
----> 5                         'https://search.naver.com/search.naver?query=무역전쟁'))
      6 connection.commit()

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\cursors.py in execute(self, query, args)
    168         query = self.mogrify(query, args)
    169 
--> 170         result = self._query(query)
    171         self._executed = query
    172         return result

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\cursors.py in _query(self, q)
    326         self._last_executed = q
    327         self._clear_result()
--> 328         conn.query(q)
    329         self._do_get_result()
    330         return self.rowcount

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\connections.py in query(self, sql, unbuffered)
    515                 sql = sql.encode(self.encoding, 'surrogateescape')
    516         self._execute_command(COMMAND.COM_QUERY, sql)
--> 517         self._affected_rows = self._read_query_result(unbuffered=unbuffered)
    518         return self._affected_rows
    519 

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\connections.py in _read_query_result(self, unbuffered)
    730         else:
    731             result = MySQLResult(self)
--> 732             result.read()
    733         self._result = result
    734         if result.server_status is not None:

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\connections.py in read(self)
   1073     def read(self):
   1074         try:
-> 1075             first_packet = self.connection._read_packet()
   1076 
   1077             if first_packet.is_ok_packet():

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\connections.py in _read_packet(self, packet_type)
    682 
    683         packet = packet_type(buff, self.encoding)
--> 684         packet.check_error()
    685         return packet
    686 

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\protocol.py in check_error(self)
    218             errno = self.read_uint16()
    219             if DEBUG: print("errno =", errno)
--> 220             err.raise_mysql_exception(self._data)
    221 
    222     def dump(self):

~\anaconda3\envs\cpu_env\lib\site-packages\pymysql\err.py in raise_mysql_exception(data)
    107         errval = data[3:].decode('utf-8', 'replace')
    108     errorclass = error_map.get(errno, InternalError)
--> 109     raise errorclass(errno, errval)

InternalError: (1364, "Field 'data_id' doesn't have a default value")
```

 과거에는 위의 로직을 전부 다 함수로 만들었다. 반드시 DB에 있는 순서로 들어가야 한다. 이 테이블의 구조가 바뀌면, 우리가 만든 함수가 작동하지 않을 수 있음을 의미한다. 유지보수에 비용이 많이 든다는 의미다.
 
 그런데 넣어야 할 데이터가 너무 많으면, 이런 문제가 생길 수 있어서, 최근에는 ORM으로 한다. 프로그램 언어 단에서 sql을 객체지향형으로 관리하자는 취지이다.
 
<pseudo-code>
    
```
# ORM 의사코드 python ver. 
class CrawlData(sa.Model):
    data_id = sa.Integer(primary=True, auto_increment=True)
    content = sa.Text()
    source_id = sa.ForeignKey

# DB 명령어
python sa migrate()
```
    
---
    
    
 조회 시 다음과 같은 명령어를 사용한다.
    
```
'SELECT * FROM data WHERE a=1' # sql문

# ORM에서는
CrawlData.query.filter(data_id=3).all()
# select * from data where data_id=3
```